# Redes Neuronales Artificales

 **Instalar Theano:**
 
 * pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
 
 
 **Instalar Tensorflow y Keras:**
 
 * conda install -c conda-forge keras
 

## Parte 1 - Pre procesado de datos

In [2]:
# Cómo importar las librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importar el data set
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
# Codificar datos categóricos
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [5]:
#El OneHotEncoder en las nuevas versiones está OBSOLETO
#onehotencoder = OneHotEncoder(categorical_features=[1])
#X = onehotencoder.fit_transform(X).toarray()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",        # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]            # Las columnas a transformar.
         )
    ], remainder='passthrough'
)

X = transformer.fit_transform(X)
X = X[:, 1:]

In [6]:
# Dividir el data set en conjunto de entrenamiento y conjunto de testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [7]:
# Escalado de variables
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Parte 2 - Construir la RNA

In [8]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
# Inicializar la RNA
classifier = Sequential()

In [10]:
# Añadir las capas de entrada y primera capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))
# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

In [11]:
# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])


In [12]:
# Ajustamos la RNA al Conjunto de Entrenamiento
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4806 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4269 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 171us/step - loss: 0.4213 - accuracy: 0.8048
Epoch 4/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4182 - accuracy: 0.8244
Epoch 5/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4155 - accuracy: 0.8280
Epoch 6/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4140 - accuracy: 0.8315
Epoch 7/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.4127 - accuracy: 0.8316
Epoch 8/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.4115 - accuracy: 0.8347
Epoch 9/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4101 - accuracy: 0.8335
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 1s 104us/step - loss: 0.4010 - accuracy: 0.8356
Epoch 79/100
8000/8000 [==============================] - 1s 96us/step - loss: 0.4009 - accuracy: 0.8342
Epoch 80/100
8000/8000 [==============================] - 1s 99us/step - loss: 0.4012 - accuracy: 0.8361
Epoch 81/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4010 - accuracy: 0.8355
Epoch 82/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4011 - accuracy: 0.8356
Epoch 83/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4010 - accuracy: 0.8345
Epoch 84/100
8000/8000 [==============================] - 1s 96us/step - loss: 0.4004 - accuracy: 0.8353
Epoch 85/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.4013 - accuracy: 0.8365
Epoch 86/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4009 - accuracy: 0.8359
Epoch 87/100
8000/8000 [==============================] - 1s 96

## Parte 3 - Evaluar el modelo y calcular predicciones finales

In [12]:
# Predicción de los resultados con el Conjunto de Testing
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

# Predecir una nueva observación

Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:

*   Geografia: Francia
*   Puntaje de crédito: 600
*   Género masculino
*   Edad: 40 años de edad
*   Tenencia: 3 años.
*   Saldo: $ 60000

*   Número de productos: 2
*   ¿Este cliente tiene una tarjeta de crédito? Sí
*   ¿Es este cliente un miembro activo? Sí
*   Salario estimado: $ 50000

Entonces, ¿deberíamos decir adiós a ese cliente?

In [1]:
new_prediction = classifier.predict(sc_X.transform(np.array([[0,0,600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

# Elaborar una matriz de confusión
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print((cm[0][0]+cm[1][1])/cm.sum())


NameError: name 'classifier' is not defined